<a href="https://colab.research.google.com/github/FVaisfeld/ml/blob/main/data_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check for bias between chinese and our data

In this lab, I will check if we need to do some fine tuning or if we can just use the chinese data for training an arrythia model on out data. 



In [3]:
# Install this package to use Colab's GPU for training
#!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Run the code below to load the dataset `Chinese_Cardisio_Combined.mat` 




---



In [39]:
#annots.keys()
#import numpy as np
#import h5py
#f = h5py.File('/content/Meta_Data.mat','r')
#f.keys()

import tables
file = tables.open_file('/content/Measurements.mat')

#import scipy.io
#mat = scipy.io.loadmat('/content/Meta_Data.mat')
#print(mat)
#mat.keys()

#data = np.array(data) # For converting to a NumPy array
#annots[‘annotations’][0][0][‘bbox_x1’], annots[‘annotations’][0][0][‘fname’]
#data = [[row.flat[0] for row in line] for line in annots[‘annotations’][0]]
#columns = [‘bbox_x1’, ‘bbox_y1’, ‘bbox_x2’, ‘bbox_y2’, ‘class’, ‘fname’]
#df_train = pd.DataFrame(data, columns=columns)


HDF5ExtError: ignored

You can then unzip the archive using the [zipfile](https://docs.python.org/3/library/zipfile.html) module.



In [ ]:
import zipfile

# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()